# Introduction

what is this course and what are we doing?

why deep learning // deep learning performance improve with [1] more data [2] size of network / bigger models (other methods plateau), [3] more computation

# The scenario

Considering house prices again.  mathematical model to predict house price based on number of beds, baths.

one of the simplest models we can use works as follows ... there's some baseline price.  then, each bedroom adds more value.  each bathroom adds more value.  (show the equation)

we'll represent this with the following picture ...

(show thing that looks like neural network)

in this case, the lines represent ...

but this a super simple model.  we can do better!

# Neural networks

motivate feature engineering.  like maybe ratio bedrooms/bathrooms makes sense as a feature.  we can add this as input to the model. or we can use deep learning which can do feature learning for us.

(show pic with intermediate features)

have to add relu or else it's equivalent to the model above (looks more complex, but actually can't learn more complex patterns).  relu just says: only allow positive values.

"deep" referring to number of layers. in this case, three layers. can add even more!

we'll look at how to train these in next lesson. for now, code for how to define them.

# Code

We’ll use **[TensorFlow 2](https://www.tensorflow.org/guide)** and TensorFlow's **[Keras](https://www.tensorflow.org/guide/keras)** API for this course. TensorFlow is one of the most popular tools for deep learning.  It is friendly for beginners and is also flexible enough for experts to implement novel models.

<img src=https://i.imgur.com/VB1Bkp4.png alt="TensorFlow and Keras logos" width=200px>

We construct a network with `keras.Sequential()` by passing it a list of layers.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Initialize a neural network
model = keras.Sequential(
    [
        layers.Dense(3, activation="relu", input_shape=(4,)),
        layers.Dense(2, activation="relu"),
        layers.Dense(1)
    ]
)

In the example above, the list has three elements, where each corresponds to a different layer in the network.
- `layers.Dense(3, activation="relu", input_shape=(4,))` creates a layer with three units and a ReLU activation function.  Since it's the first layer in the network, we can set the size of the input layer (in this case, the network accepts an input with four units).
- `layers.Dense(2, activation="relu")` is the second layer in the network.  It has two units and a ReLU activation function.
- `layers.Dense(1)` is the final layer in the network and has one unit.

The image below visualizes this network.

<img src="https://i.imgur.com/aLCuZKg.png" width="40%"/>

You can display the contents of the network with `model.summary()`.

In [ ]:
# Display contents of the neural network
model.summary()

As printed above, this neural network has 26 parameters.

outputs one row for each layer. number of parameters is just number of weights in the model.  take the time to check that this model should have 26 parameters - does it make sense to you?

when we just create a neural network, the weights are entirely random and so will yield horrible predictions.

we'll have to train the network to detect patterns in the training data. will learn how to do that in upcoming tutorial.

# Your turn 

...